In [1]:
%%capture
%pip install -U transformers datasets accelerate peft trl bitsandbytes wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B for JUnit Test case Generation', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: walim (iHateNLP). Use `wandb login --relogin` to force relogin


In [5]:
base_model = "meta-llama/Meta-Llama-3.1-8B"
dataset_name = "CodexAI/dataset"
new_model = "CODEX-Meta-Llama-3.1-8B"

In [6]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [7]:
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    device={"":0}
    torch_type=torch.bfloat16
else:
    device="cpu"
    torch_type=torch.bfloat16
    print("I am begging for mercy already!")

CUDA device: Tesla P100-PCIE-16GB


In [8]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_type,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map=device,
    attn_implementation=attn_implementation,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-

In [ ]:
model.print_trainable_parameters()

In [10]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'up_proj', 'down_proj', 'q_proj', 'o_proj', 'gate_proj', 'v_proj', 'k_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [11]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)
model

In [ ]:
model.print_trainable_parameters()

In [12]:
dataset = load_dataset(dataset_name)
dataset

README.md:   0%|          | 0.00/426 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4687 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['focal_method', 'test_case'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['focal_method', 'test_case'],
        num_rows: 4687
    })
})

In [13]:
train=dataset['train']
train=train.shuffle(True).select(range(1000))
train

Dataset({
    features: ['focal_method', 'test_case'],
    num_rows: 1000
})

In [14]:
test=dataset['test']
test=test.shuffle(True).select(range(100))
test

Dataset({
    features: ['focal_method', 'test_case'],
    num_rows: 100
})

In [15]:
def format_chat_template(row):
    row_json = [
        {"role": "system", "content": "You are a coding assistant for generating JUnit test cases."},
        {"role": "user", "content": row["focal_method"]},
        {"role": "assistant", "content": row["test_case"]}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row


In [16]:
train = train.map(
    format_chat_template,
    num_proc= 4,
)

train

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Dataset({
    features: ['focal_method', 'test_case', 'text'],
    num_rows: 1000
})

In [17]:
test = test.map(
    format_chat_template,
    num_proc= 4,
)

test

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['focal_method', 'test_case', 'text'],
    num_rows: 100
})

In [18]:
train['text'][3]

'<|im_start|>user\npublic OutputStream createRawOutputStream() throws IOException\n    {\n        checkClosed();\n        if (isWriting)\n        {\n            throw new IllegalStateException("Cannot have more than one open stream writer.");\n        }\n        if (randomAccess != null)\n            randomAccess.clear();\n        else\n            randomAccess = getStreamCache().createBuffer();\n        OutputStream out = new RandomAccessOutputStream(randomAccess);\n        isWriting = true;\n        return new FilterOutputStream(out)\n        {\n            @Override\n            public void write(byte[] b, int off, int len) throws IOException\n            {\n                this.out.write(b, off, len);\n            }\n            \n            @Override\n            public void close() throws IOException\n            {\n                super.close();\n                setInt(COSName.LENGTH, (int)randomAccess.length());\n                isWriting = false;\n            }\n        };\n 

In [19]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    run_name ="./loggings",
    overwrite_output_dir=True,
    eval_strategy="steps",
    eval_steps=0.10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    warmup_steps=10,
    logging_strategy="steps",
    logging_steps=50,
    logging_first_step=True,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
)

In [20]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [30]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [31]:
trainer.eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 100
})

In [21]:
trainer.train()

Step,Training Loss,Validation Loss
50,1.371500,1.268404
100,1.392400,1.265884
150,1.266900,1.245193
200,1.410900,1.233470
250,1.302100,1.223866
300,1.415900,1.220015
350,1.223900,1.212525
400,1.306600,1.210337
450,1.307100,1.203429
500,1.199800,1.201871


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=500, training_loss=1.267729706287384, metrics={'train_runtime': 4208.3025, 'train_samples_per_second': 0.238, 'train_steps_per_second': 0.119, 'total_flos': 1.1397699521519616e+16, 'train_loss': 1.267729706287384, 'epoch': 1.0})

In [22]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128258, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [23]:
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


eval/loss,██▆▄▃▃▂▂▁▁
eval/runtime,█▄▆▄▁▄▄▄▅▄
eval/samples_per_second,▁▁▁▁▁▁▁▁▁▁
eval/steps_per_second,▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇████
train/grad_norm,▂▂▃▂▄▂▅▁▅▃▃▂▅▂▄▃▄▃▄▁█
train/learning_rate,▂██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▆▇▂▇▃▄▂█▃▅▁█▁▃▄▅▁▅▁▂
eval/loss,1.20187
eval/runtime,137.9093


In [25]:
messages = [{"role": "user", "content": """
public class SimpleCalculator {
    // Method to add two numbers
    public int add(int a, int b) {
        return a + b;
    }

    // Method to subtract two numbers
    public int subtract(int a, int b) {
        return a - b;
    }

    // Method to multiply two numbers
    public int multiply(int a, int b) {
        return a * b;
    }

    // Method to divide two numbers
    // Throws ArithmeticException if divisor is zero
    public double divide(int a, int b) {
        if (b == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return (double) a / b;
    }
}
"""}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


@Test
    public void testAdd() {
        assertEquals(5, calculator.add(2, 3));
    }


In [26]:
from huggingface_hub import HfApi, create_repo

In [27]:
repo_name = new_model
organization_name = "CodexAI"
repo_url = f"{organization_name}/{repo_name}"

In [28]:
create_repo(repo_url, repo_type="model", private=True,exist_ok=True)

RepoUrl('https://huggingface.co/CodexAI/CODEX-Meta-Llama-3.1-8B', endpoint='https://huggingface.co', repo_type='model', repo_id='CodexAI/CODEX-Meta-Llama-3.1-8B')

In [29]:
api = HfApi()
api.upload_folder(folder_path=new_model,repo_id=repo_url)

adapter_model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CodexAI/CODEX-Meta-Llama-3.1-8B/commit/acee6c73fad1c5e63e5d56c0abbd60c592fcd2ed', commit_message='Upload folder using huggingface_hub', commit_description='', oid='acee6c73fad1c5e63e5d56c0abbd60c592fcd2ed', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
print('END')

END
